In [27]:
import pandas as pd
import numpy as np

dataset_dir = "../datasets/ml-25m"

movies_df = pd.read_csv(dataset_dir + "/movies.csv")
movies_df = movies_df.set_index("movieId")
ratings_df = pd.read_csv(dataset_dir + "/ratings.csv")
tags_df = pd.read_csv(dataset_dir + "/tags.csv")

print('Setup complete!')

Setup complete!


In [28]:
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
209157,We (2018),Drama
209159,Window of the Soul (2001),Documentary
209163,Bad Poems (2018),Comedy|Drama


In [29]:
ratings_df = ratings_df.drop("timestamp", axis=1)
ratings_df

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


In [30]:
def add_valid_movies(row):
    if row["counts"] >= 5:
        movieIds.add(row["movieId"])

movieIds = set()
movie_rating_counts = ratings_df["movieId"].value_counts().rename_axis('movieId').reset_index(name='counts')
movie_rating_counts.apply(add_valid_movies, axis=1)
print(len(movieIds))

32720


In [31]:
movies_df = movies_df[movies_df.index.map(lambda id: id in movieIds)]
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
208385,Holiday in the Wild (2019),Drama|Romance
208715,Let It Snow (2019),Comedy|Romance
208737,Midway (2019),Action|Drama|War


In [32]:
genres_list = []
for index, row in movies_df.iterrows():
    try:
        genres = row['genres'].split("|")
        genres_list.extend(genres)
    except:
        genres_list.append(row["genres"])

genres_list = list(set(genres_list))

In [33]:
genres_list.remove("IMAX")
repl_idx = genres_list.remove("(no genres listed)")
for genre in genres_list: 
    print(genre)
print(len(genres_list))

Adventure
Fantasy
Drama
Children
Comedy
Sci-Fi
Film-Noir
Horror
War
Animation
Western
Romance
Crime
Musical
Mystery
Thriller
Action
Documentary
18


In [34]:
movies_df1 = movies_df.copy()
for genre in genres_list:
    movies_df1[genre] = movies_df1["genres"].map(lambda o: 1 if genre in str(o) else 0)
movies_df1

,title,genres,Adventure,Fantasy,Drama,Children,Comedy,Sci-Fi,Film-Noir,Horror,War,Animation,Western,Romance,Crime,Musical,Mystery,Thriller,Action,Documentary
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Jumanji (1995),Adventure|Children|Fantasy,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208385,Holiday in the Wild (2019),Drama|Romance,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
208715,Let It Snow (2019),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
208737,Midway (2019),Action|Drama|War,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [35]:
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
208385,Holiday in the Wild (2019),Drama|Romance
208715,Let It Snow (2019),Comedy|Romance
208737,Midway (2019),Action|Drama|War


In [36]:
tags_df = tags_df[tags_df["movieId"].map(lambda id: id in movieIds)]

In [38]:
tags_df["tag"] = tags_df["tag"].map(lambda t: str(t).lower()) #all tags to small letters
tag_counts = tags_df["tag"].value_counts().rename_axis('tag').reset_index(name='counts')
tag_counts = tag_counts.groupby("tag").counts.agg([sum]) #merge duplicates
tag_counts

,sum
tag,
alexander skarsgård,1
ballet school,1
breakup,1
filmes antigos,2
filmes antigos,2
...,...
淘金记 莫声版,1
独闯龙潭,1
臥底,1


In [39]:
tag_counts = tag_counts[tag_counts["sum"] >= 20]
tag_counts = tag_counts.sort_values(["sum","tag"], ascending=[False,True]).reset_index()
tag_counts

,tag,sum
0,sci-fi,8762
1,atmospheric,7041
2,action,6703
3,comedy,6188
4,surreal,5540
...,...,...
5488,women's rights,20
5489,writers at work,20
5490,yahoo top pick,20
5491,young boy,20


In [40]:
init_tag = np.zeros(len(movies_df))
movies_df2 = movies_df.copy()

def add_tags_as_features(tag):
    relevant_tags.add(tag)
    movies_df2[tag] = init_tag
    
relevant_tags = set()
tag_counts["tag"].apply(add_tags_as_features)
    

0       None
1       None
2       None
3       None
4       None
        ... 
5488    None
5489    None
5490    None
5491    None
5492    None
Name: tag, Length: 5493, dtype: object

In [41]:
movies_df2 = movies_df2.drop(["genres", "title"], axis=1)
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,dystopia,...,watch-again,well-written,whiskey,william friedkin,william holden,women's rights,writers at work,yahoo top pick,young boy,yul brynner
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
#around 10 minutes

def add_tags_to_movies(row):
    tag = row["tag"]
    if tag not in relevant_tags:
        return
    movieId = row["movieId"]
    movies_df2.loc[movieId, tag] += 1
    
tags_df.apply(add_tags_to_movies, axis=1)

#for index, row in tags_df.iterrows():
#    if index % 10000 == 0:
#        print(index/10000)
#    tag = row["tag"]
#    if tag not in relevant_tags:
#        continue
#    movieId = row["movieId"]
#    movies_df.loc[movieId, tag] += 1

0          None
1          None
2          None
3          None
4          None
           ... 
1093352    None
1093353    None
1093354    None
1093355    None
1093356    None
Length: 1035024, dtype: object

In [43]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,dystopia,...,watch-again,well-written,whiskey,william friedkin,william holden,women's rights,writers at work,yahoo top pick,young boy,yul brynner
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
movies_df2["funny"]

movieId
1         30.0
2          0.0
3          1.0
4          0.0
5          0.0
          ... 
208385     0.0
208715     0.0
208737     0.0
208747     0.0
208939     0.0
Name: funny, Length: 32720, dtype: float64

In [45]:
movies_df2["max"] = movies_df2.max(axis=1)
novi = movies_df2[movies_df2["max"] > 0]
novi = novi.div(novi["max"], axis=0)
novi = novi.drop("max", axis=1)
novi

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,dystopia,...,watch-again,well-written,whiskey,william friedkin,william holden,women's rights,writers at work,yahoo top pick,young boy,yul brynner
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.008772,0.0,0.008772,0.175439,0.0,0.263158,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.058824,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.0,0.500000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207888,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208096,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208295,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,dystopia,...,well-written,whiskey,william friedkin,william holden,women's rights,writers at work,yahoo top pick,young boy,yul brynner,max
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
movies_df2 = novi
movies_df1 = movies_df1.drop(["genres", "title"], axis=1)
movies_df1

,Adventure,Fantasy,Drama,Children,Comedy,Sci-Fi,Film-Noir,Horror,War,Animation,Western,Romance,Crime,Musical,Mystery,Thriller,Action,Documentary
movieId,,,,,,,,,,,,,,,,,,
1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208385,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
208715,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
208737,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [48]:
movies_df = pd.concat([movies_df1, movies_df2], axis=1)
movies_df = movies_df.dropna()
movies_df

,Adventure,Fantasy,Drama,Children,Comedy,Sci-Fi,Film-Noir,Horror,War,Animation,...,watch-again,well-written,whiskey,william friedkin,william holden,women's rights,writers at work,yahoo top pick,young boy,yul brynner
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,1,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207888,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208096,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208295,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
#few minutes
movies_df.to_csv("movie_features1.csv")

In [50]:
ratings_df

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


In [51]:
moviesIds = set(movies_df.index)
len(moviesIds)

27832

In [52]:
ratings_df = ratings_df[ratings_df["movieId"].map(lambda id: id in moviesIds)]
ratings_df

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


In [53]:
ratings_df.to_csv("rating_features1.csv")